# Create Slice for Fabric Rack
This is the slice creation for the ```Ping``` tutorial. This is the first step of the assignment, to create the slice on fabric using fablib.
 

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 2 nodes for the Ping Assignment.
### 1.1 Reserve Resources
In this section we will declare the structure of the slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "MyLayer2PingSlice"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    #Create Slice
    slice = fablib.new_slice(slice_name)

    # Add node
    client = slice.add_node(name="client", site=site)
    client.set_capacities(cores=cores, ram=ram, disk=disk)
    client.set_image(image)
    iface1 = client.add_component(model='NIC_Basic', name="client-nic1").get_interfaces()[0]
    
    # Add node
    server = slice.add_node(name="server", site=site)
    server.set_capacities(cores=cores, ram=ram, disk=disk)
    server.set_image(image)
    iface2 = server.add_component(model='NIC_Basic', name="server-nic1").get_interfaces()[0] 
    
    # Network
    net1 = slice.add_l2network(name="bridge1", interfaces=[iface1, iface2])

    #Submit Slice Request
    slice.submit()
    
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
This section of the slice creation, once the nodes are active, will assign the network spaces some predefined values that we will use for the Assignment.

In [ ]:
# Setup Network
try:
    client_name = "client"
    server_name = "server"
    network_service_name = "bridge1"

    client_ip = '10.20.30.40'
    server_ip = '10.20.30.41'
    
    client = slice.get_node(name=client_name)        
    iface1 = client.get_interface(network_name=network_service_name)  
    iface1.set_ip(ip=client_ip, cidr="24")
    
    iface1.get_device_name()
    
    client.execute("echo 100.20.30.40 client | sudo tee -a /etc/hosts")
    client.execute(f"sudo ip link set dev { iface1.get_device_name()} up")
    
    stdout, stderr = client.execute(f'ip addr show {iface1.get_os_interface()}')
    server = slice.get_node(name=server_name)        
    iface2 = server.get_interface(network_name=network_service_name)  
    iface2.set_ip(ip=server_ip, cidr="24")
    
    stdout, stderr = server.execute(f'ip addr show {iface2.get_os_interface()}')
    server.execute(f"sudo ip link set dev { iface2.get_device_name()} up")
    
    Server.execute("echo 200.20.30.40 server | sudo tee -a /etc/hosts")
except Exception as e:
    print(f"Error: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
In this section of the slice creation, we will add any additional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    print("No additional software Needed")
except Exception as e:
    print(f"Exception: {e}")

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the Ping Notebook. You can either open it on the explorer or click [Here](./ping.ipynb) to open the next notebook.